# comment un perceptron est-il capable de reconnaitre un chiffre ?

Dans cet exercice, nous allons créer les données d'entrée pour un réseau de neurones (un simple perceptron pour l'instant), puis faire en sorte que ce neurone nous dise qu'il a reconnu le chiffre.

L'objectif est de :
1. Créer une grille vide de 28x28 pixels.
2. Dessiner un chiffre '1' simple dedans.
3. Visualiser cette grille.
4. Transformer cette grille 2D en une longue liste de 784 valeurs, qui seront les entrées de notre perceptron.
5. mettre en place le module z pour calculer son activation.
6. afficher le chiffre auquel il correspond.

In [7]:
import numpy as np
import matplotlib.pyplot as plt

## Étape 1 : Création de la matrice vide

Nous commençons par créer une matrice de taille 28x28 remplie de zéros. Chaque '0' représente un pixel noir (vide).

In [8]:
# Création d'une matrice 28x28 remplie de 0
matrice = np.zeros((28, 28))

print("Forme de la matrice :", matrice.shape)
print("\nAperçu de la matrice (remplie de 0) :\n", matrice)

## Étape 2 : Dessiner un chiffre

Nous avons créé une fonction spéciale `draw_digit` qui permet de dessiner des chiffres "digitaux" (style réveil) dans notre matrice.

**Changez la valeur de `chiffre` ci-dessous pour tester avec 0, 1, 2, ..., 9 !**

In [9]:
# Fonction pour dessiner un chiffre (style 7 segments)
def draw_digit(mat, digit):
    # Réinitialisation de la matrice
    mat.fill(0)
    
    # Coordonnées des segments (approximatif pour 28x28)
    # A: Haut, B: Haut-Droit, C: Bas-Droit, D: Bas, E: Bas-Gauche, F: Haut-Gauche, G: Milieu
    seg_a = (slice(2, 4), slice(8, 20))
    seg_b = (slice(4, 14), slice(18, 20))
    seg_c = (slice(14, 24), slice(18, 20))
    seg_d = (slice(24, 26), slice(8, 20))
    seg_e = (slice(14, 24), slice(8, 10))
    seg_f = (slice(4, 14), slice(8, 10))
    seg_g = (slice(13, 15), slice(9, 19))
    
    segments = {
        0: [seg_a, seg_b, seg_c, seg_d, seg_e, seg_f],
        1: [seg_b, seg_c], # Le 1 est souvent à droite
        2: [seg_a, seg_b, seg_g, seg_e, seg_d],
        3: [seg_a, seg_b, seg_g, seg_c, seg_d],
        4: [seg_f, seg_g, seg_b, seg_c],
        5: [seg_a, seg_f, seg_g, seg_c, seg_d],
        6: [seg_a, seg_f, seg_g, seg_e, seg_d, seg_c],
        7: [seg_a, seg_b, seg_c],
        8: [seg_a, seg_b, seg_c, seg_d, seg_e, seg_f, seg_g],
        9: [seg_a, seg_b, seg_c, seg_d, seg_f, seg_g]
    }
    
    if digit == 1:
        # Petit ajustement pour le 1 : on le centre pour faire joli comme avant
        mat[5:23, 14] = 1
        return

    if digit in segments:
        for seg in segments[digit]:
            mat[seg] = 1

# --- CHOISISSEZ VOTRE CHIFFRE ICI ---
# Changez ce nombre (0-9) et relancez les cellules !
chiffre = int(input("Quel chiffre voulez-vous (0-9) ? "))
# ------------------------------------
# Pour demander à l'utilisateur : 
# chiffre = int(input("Quel chiffre voulez-vous (0-9) ? "))

print(f"Dessin du chiffre {chiffre}...")
draw_digit(matrice, chiffre)

## Étape 3 : Visualisation

Regardons à quoi ressemble notre matrice sous forme d'image.

In [10]:
plt.imshow(matrice, cmap='gray')
plt.title(f"Représentation visuelle du chiffre {chiffre} (28x28)")
plt.colorbar()
plt.show()

## Étape 4 : Aplatissement (Flattening) pour le Perceptron

Un perceptron classique prend généralement ses entrées sous forme d'un vecteur (une liste à une dimension), et non une grille 2D.

Nous devons donc "aplatir" notre image :
28 lignes * 28 colonnes = 784 pixels au total.

Nos 784 valeurs deviendront les 784 entrées ($x_1, x_2, ..., x_{784}$) de notre neurone.

In [11]:
# Aplatissement de la matrice
inputs = matrice.flatten()

print("Nouvelle forme des données :", inputs.shape)
print("Nombre total d'entrées pour le neurone :", len(inputs))

pour reconnaitre un chiffre, il faut que le réseau de neurones soit capable de les reconnaitre.

pour cela, nous allons utiliser les biais de chaque neurone, car les biais sont les poids des liaisons des entrées sur le neurone, donc l'idée est de dire que selon les pixels de l'image, ce sont tels et tels pixels qui nous intéressent, et pas les autres.
les poids sont des simples nombres (-2, 1, 2, etc)
le neurone quant à lui, va faire la somme des poids des entrées multipliée par valeurs des entrées, et si la somme est supérieure à un certain seuil, le neurone va s'activer, sinon il restera inactif.

$$z=w_1a_1 + w_2a_2 + ... + w_na_n$$
$$z=\sum_{i=1}^{n} w_i a_i$$

calcul de z

In [12]:
def calculate_liear_combination(inputs, weights, bias) :
    output = 0
    for i in range (len(inputs)) :
        output += inputs[i] * weights[i]
    output += bias
    return output

création du perceptron

In [13]:
def first_neuron(inputs, digit_label):
    print(f"--- DÉBUT DU NEURONE RECONNAISSEUR DE '{digit_label}' ---")
    
    # 1. Les Données
    neuron_inputs = inputs
    
    # 2. Les Poids (Apprentissage)
    weights = []
    for i in range(len(neuron_inputs)):
        if neuron_inputs[i] > 0:
            weights.append(1.0)
        else:
            weights.append(-0.1)
            
    # 3. Le Biais (Calibration)
    score_max = 0
    for i in range(len(neuron_inputs)):
        score_max += neuron_inputs[i] * weights[i]
    biais = -score_max + 0.1

    # 4. Le Calcul
    output = calculate_liear_combination(neuron_inputs, weights, biais)
    
    # 5. L'Activation (La Décision)
    # Si z > 0, le neurone s'active.
    print(f"Biais calculé : {biais:.2f}")
    print(f"Résultat du calcul (z) : {output:.2f}")
    
    is_active = output > 0
    result_text = f"✅ RECONNU : {digit_label}" if is_active else f"❌ NON RECONNU : {digit_label}"
    color_text = 'green' if is_active else 'red'
    
    # --- VISUALISATION TABLEAU DE BORD ---
    plt.figure(figsize=(15, 5))
    
    # --- VUE 1 : L'ENTRÉE (MATRICE 28x28) ---
    plt.subplot(1, 4, 1)
    plt.imshow(neuron_inputs.reshape(28, 28), cmap='gray')
    plt.title("1. Matrice d'Entrée")
    plt.axis('off')
    
    # --- VUE 2 : LES POIDS (APERCU VECTORIEL) ---
    plt.subplot(1, 4, 2)
    # On crée une fausse "image" pour montrer les poids
    # C'est juste pour visualiser la liste verticale
    # On affiche les 3 premiers et 3 derniers
    weights_display = np.zeros((10, 1))
    weights_display[0:3, 0] = weights[0:3]
    # Milieu vide (0)
    weights_display[7:10, 0] = weights[-3:]
    
    plt.imshow(weights_display, cmap='coolwarm', aspect='auto')
    plt.title(f"2. Poids (784)\n[w0..w2]\n...\n[w781..w783]")
    plt.yticks([])
    plt.xticks([])
    # Annotations textuelles sur le plot
    plt.text(0, 1, f"{weights[0]:.1f}", ha='center', color='black', fontweight='bold')
    plt.text(0, 8, f"{weights[-1]:.1f}", ha='center', color='black', fontweight='bold')
    
    # --- VUE 3 : LE CALCUL (BIAIS & SOMME) ---
    plt.subplot(1, 4, 3)
    plt.axis('off')
    plt.title("3. Calcul")
    plt.text(0.1, 0.8, f"Somme Pondérée :\n{score_max:.2f}", fontsize=12)
    plt.text(0.1, 0.5, f"Biais :\n{biais:.2f}", fontsize=12, color='blue')
    plt.text(0.1, 0.2, f"Total (Z) :\n{output:.2f}", fontsize=14, fontweight='bold')
    
    # --- VUE 4 : RESULTAT ---
    plt.subplot(1, 4, 4)
    plt.axis('off')
    plt.title("4. Résultat")
    plt.text(0.5, 0.5, result_text, ha='center', va='center', fontsize=16, color=color_text, fontweight='bold', wrap=True)
    
    plt.tight_layout()
    plt.show()

    return output

In [ ]:
# On lance le neurone en lui disant quel chiffre il est censé reconnaitre (celui qu'on a choisi)
first_neuron(inputs, chiffre)

## Étape 5 : La Sortie (Output)

Nous avons nos entrées. Maintenant, parlons de la **sortie**.

Comme nous voulons reconnaitre des chiffres manuscrits allant de 0 à 9, nous avons **10 possibilités**.
Il est donc logique d'avoir une couche de sortie composée de **10 neurones**.

- Le neurone 0 s'allume si l'image est un 0.
- Le neurone 1 s'allume si l'image est un 1.
- ...
- Le neurone 9 s'allume si l'image est un 9.

### Objectif pour notre image
Puisque nous avons dessiné un **1**, nous voulons que le réseau de neurones active la sortie correspondant au "1" et désactive toutes les autres.

In [ ]:
# Définition de la sortie attendue (Target)
# C'est un vecteur de 10 valeurs [0-9]
target = np.zeros(10)

# On veut que la case correspondante au chiffre dessiné soit à 1
target[chiffre] = 1

print(f"Sortie attendue (Target) pour le chiffre {chiffre} :")
print(target)

Ce vecteur représente la **vérité terrain**.

Toute la mission du réseau de neurones sera d'ajuster ses poids pour que, en recevant les 784 pixels de notre image en entrée, il produise une sortie qui ressemble le plus possible à ce vecteur `target`.